In [1]:
# Disable TensorFlow debugging info and warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # 2: Info and warnings not displayed 

In [2]:
import tensorflow as tf
import numpy as np
import skimage.io
import sklearn.decomposition
import matplotlib.pyplot as plt
import pathlib
import tqdm
import pickle
import annotation, misc, hyspec_cnn, hyspec_io, hyspec_ml, preprocess
import datetime

In [3]:
# Disable GPUs
tf.config.set_visible_devices([], 'GPU')
visible_devices = tf.config.get_visible_devices()
for device in visible_devices:
    assert device.device_type != 'GPU'
print(tf.version.VERSION)

2.12.0


In [4]:
# Paths
hyspec_base = pathlib.Path('/media/mha114/Massimal/Vega_Sola/Hyperspectral/20220823/Area/2a_Rrs')
annotation_dir = pathlib.Path('/media/mha114/Massimal/Vega_Sola/Hyperspectral/20220823/Area/3a_PCA_TrainValidationSplit/Training/Annotation')
water_mask_dir = pathlib.Path('/media/mha114/Massimal/Vega_Sola/Hyperspectral/20220823/Area/M_Annotation_WaterMask/20220823_VegaSola_WaterMask_Gray')
pca_dir = pathlib.Path('/media/mha114/Massimal/Vega_Sola/Hyperspectral/20220823/Area/M_PCAModel')
pca_model_path = pca_dir / '20220823_Vega_Sola_PCA-Model.npz'
pca_im_dir = pathlib.Path('/media/mha114/Massimal/Vega_Sola/Hyperspectral/20220823/Area/3a_PCA_TrainValidationSplit/Training/PCA-Images')

In [5]:
label_image_paths = [path for path in sorted(annotation_dir.glob('*.png'))]
for path in label_image_paths: print(path.stem)

Vega_SolaNW_Coast1_Pika_L_10-Crop Wavelengths.bip
Vega_SolaNW_Coast1_Pika_L_11-Crop Wavelengths.bip
Vega_SolaNW_Coast1_Pika_L_12-Crop Wavelengths.bip
Vega_SolaNW_Coast1_Pika_L_13-Crop Wavelengths.bip
Vega_SolaNW_Coast1_Pika_L_16-Crop Wavelengths.bip
Vega_SolaNW_Coast1_Pika_L_18-Crop Wavelengths.bip
Vega_SolaNW_Coast1_Pika_L_22-Crop Wavelengths.bip
Vega_SolaNW_Coast1_Pika_L_6-Crop Wavelengths.bip
Vega_SolaNW_Coast1_Pika_L_7-Crop Wavelengths.bip
Vega_SolaNW_Coast1_Pika_L_8-Crop Wavelengths.bip
Vega_SolaNW_Coast1_Pika_L_9-Crop Wavelengths.bip
Vega_SolaNW_Coast2_Pika_L_14-Crop Wavelengths.bip
Vega_SolaNW_Coast2_Pika_L_16-Crop Wavelengths.bip
Vega_SolaNW_Coast2_Pika_L_17-Crop Wavelengths.bip
Vega_SolaNW_Coast2_Pika_L_19-Crop Wavelengths.bip
Vega_SolaNW_Coast2_Pika_L_20-Crop Wavelengths.bip
Vega_SolaNW_Coast2_Pika_L_21-Crop Wavelengths.bip
Vega_SolaNW_Coast2_Pika_L_22-Crop Wavelengths.bip
Vega_SolaNW_Coast2_Pika_L_23-Crop Wavelengths.bip
Vega_SolaNW_Coast2_Pika_L_24-Crop Wavelengths.bip
Vega

In [6]:
# Load PCA model 
W_pca,X_mean,X_std = hyspec_ml.read_pca_model(pca_model_path)

In [7]:
# Loop over images, PCA transform, inpaint, save
for label_image_path in tqdm.tqdm(label_image_paths):
    hyspec_image_path = [path for path in hyspec_base.rglob(label_image_path.stem+'*.hdr')][0]
    water_mask_path = [path for path in water_mask_dir.rglob(label_image_path.stem+'*.png')][0]

    # Read images
    label_image = skimage.io.imread(label_image_path)
    water_mask_image = np.expand_dims(skimage.io.imread(water_mask_path),axis=2)
    (hyspec_image,_,_,_) = hyspec_io.load_envi_image(str(hyspec_image_path))
    hyspec_image_masked = hyspec_image*water_mask_image

    # Create PCA image
    im_pca = hyspec_ml.pca_transform_image(hyspec_image_masked,W_pca,X_mean,X_std)
    
    # Inpaint PCA image
    im_pca_inpaint = preprocess.inpaint_missing_pixels(im_pca)
    
    # Set labels for zero-valued pixels to background
    label_image[np.all(im_pca_inpaint,axis=2)==0] = 0
    
    # Save as tensorflow dataset
    dataset = tf.data.Dataset.from_tensors((im_pca_inpaint,label_image))  # Use tf.data.Dataset.concatenate to merge later
    dataset_save_path = pca_im_dir / label_image_path.stem
    dataset.save(str(dataset_save_path))


100%|███████████████████████████████████████████| 71/71 [29:06<00:00, 24.59s/it]
